# Retrieve embedding useage

### Using ElasticSearchVectorStore and Embedding

In [ ]:
from langchain_community.vectorstores.elasticsearch import ElasticsearchStore
from langchain_community.embeddings import LocalAIEmbeddings
from elasticsearch import Elasticsearch
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
# 환경 변수 로딩 
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
es_hosts = os.getenv('ES_HOST')
es_id = os.getenv('ES_ID')
es_pwd = os.getenv('ES_PWD')
index_name = os.getenv('ES_INDEX_NAME')

base_url = os.getenv("EMBEDDING_BASE_URL")
model = os.getenv("EMBEDDING_MODEL")

dataset = pd.DataFrame()
batch_count = 10

### embedding 클래스 정의

In [ ]:
embeddings = LocalAIEmbeddings(openai_api_base=base_url, 
                                       model=model,
                                       openai_api_key=OPENAI_API_KEY)

### 예제용 환경 변수 

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=250)

텍스트 길이가 길기 때문에 RecursiveCharacterTextSplitter 사용 청크 단위로 잘라서 임베딩 <br> 청크와의 오버래핑을 250으로 지정

In [ ]:
es = ElasticSearch([es_hosts], basic_auth=(es_id,es_pwd))

es 인스턴스 생성

In [ ]:
vectorstore = ElasticsearchStore(
            index_name=index_name,
            embedding=embeddings,
            es_connection=es
        )

vectorstore 인스턴스 생성 

In [ ]:
for offset in tqdm(range(0, len(dataset), batch_count)):
    loader = DataFrameLoader(dataset[offset:offset+batch_count], page_content_column="content")
    all_splits = text_splitter.split_documents(loader.load())
    vectorstore.add_documents(all_splits)
    del all_splits
    del loader
                
es.index(index=settings.NEWS_BULK_INDEX, body={"flush": "true"})
es.close()

dataset 데이터 프레임을 이용하여 전체 데이터에 대해서 임베딩을 수행하고 <br>vectorstore에 문서 입력.<br> 루프 바깥의 index flush는 입력된 문서를 ES에 즉시 적용